# Import statements

In [1]:
from pytao import Tao
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.patches as mpatches
import re
import io
from os import path,environ
import pandas as pd
import random
from IPython.display import display, clear_output, update_display
import bayes_opt

from pmd_beamphysics import ParticleGroup
#from pmd_beamphysics.statistics import resample_particles
import pmd_beamphysics.statistics

from UTILITY_plotNMM import plotNMM, slicePlotNMM
from UTILITY_linacPhaseAndAmplitude import getLinacMatchStrings, setLinacPhase, setLinacGradientAuto
from UTILITY_modifyAndSaveInputBeam import modifyAndSaveInputBeam
from UTILITY_setLattice import setLattice, getBendkG, getQuadkG, getSextkG

import os
filePath = os.getcwd()
%env FACET2_LATTICE {filePath}
#%env FACET2_LATTICE /Users/nmajik/Documents/SLAC/slacsyncgit/bmadExample

print('Make sure enviornment was set!:  ', environ['FACET2_LATTICE']) 

env: FACET2_LATTICE=/Users/nmajik/Documents/SLAC/FACET2-Bmad-PyTao
Make sure enviornment was set!:   /Users/nmajik/Documents/SLAC/FACET2-Bmad-PyTao


# Start tao

In [2]:
tao=Tao('-init {:s}/bmad/models/f2_elec/tao.init -noplot'.format(environ['FACET2_LATTICE'])) 
#tao=Tao('-init {:s}/bmad/models/f2_elec/tao.init'.format(environ['FACET2_LATTICE'])) 

#Need to explicitly add non-MARKER places to save beam...
tao.cmd("set beam add_saved_at = DTOTR, XTCAVF, M2EX")

[]

In [3]:
#Compare Lucretia vs Bmad DTOTR
bmadZOffset = 2015.26 - 1016.16

## Test 10 GeV imaging case

In [4]:
setLattice(tao, Q0DkG = -115.82, Q1DkG = 180.69, Q2DkG = -115.82)
zMin = 1993.91 - bmadZOffset
zMax = 2015.26 - bmadZOffset

tao.cmd(f"show taylor_map -s {zMin} {zMax}")

['Symplectic Error:  1.22E-15',
 '',
 '    -3.50939072    -0.00970279     0.00000000     0.00000000     0.00000000    -0.00000000   :  5.12838631E-17',
 '    -0.29983580    -0.28577874     0.00000000     0.00000000     0.00000000     0.00000000   :  1.83691865E-31',
 '    -0.00000000     0.00000000    -1.96057888    -0.00290215     0.00000000     0.05590319   :  4.65995863E-14',
 '    -0.00000000     0.00000000    -0.13491947    -0.51025315     0.00000000     0.00600002   : -3.12690058E-19',
 '     0.00000000     0.00000000     0.00422108    -0.02850736     1.00000000    -0.00000581   :  8.34763835E-17',
 '     0.00000000     0.00000000     0.00000000     0.00000000     0.00000000     1.00000000   :  1.97215226E-30']

## Test non-10 GeV case

In [5]:
setLattice(tao, Q0DkG = -97.9, Q1DkG = 151.7, Q2DkG = -97.9)

zMin = 1994.62 - bmadZOffset
zMax = 2015.26 - bmadZOffset

tao.cmd(f"show taylor_map -s {zMin} {zMax}")

['Symplectic Error:  6.66E-16',
 '',
 '    -1.41645383     9.92918643     0.00000000     0.00000000     0.00000000    -0.00000000   :  7.19458906E-17',
 '    -0.15963666     0.41304711    -0.00000000     0.00000000     0.00000000     0.00000000   : -5.29822316E-31',
 '     0.00000000    -0.00000000    -1.78567704     5.00498902     0.00000000     0.05590320   :  2.80761426E-14',
 '     0.00000000     0.00000000    -0.14158087    -0.16318142     0.00000000     0.00600002   :  5.04538781E-20',
 '     0.00000000    -0.00000000     0.00279927    -0.03915239     1.00000000    -0.00000581   :  5.09202992E-18',
 '     0.00000000     0.00000000     0.00000000     0.00000000     0.00000000     1.00000000   :  3.15544362E-30']

In [6]:
#Now change the energy with PATCH
tao.cmd("set ele patch_sector20 e_tot_set=8e9")
tao.cmd(f"show taylor_map -s {zMin} {zMax}")

['Symplectic Error:  2.70E-15',
 '',
 '     0.04936916    12.36025798     0.00000000    -0.00000000     0.00000000     0.00000000   : -7.48561870E-12',
 '    -0.07785598     0.76322864    -0.00000000     0.00000000     0.00000000     0.00000000   : -2.30332015E-18',
 '     0.00000000    -0.00000000    -1.36069341     7.19963657     0.00000000     0.03578933   :  2.23330907E-03',
 '     0.00000000    -0.00000000    -0.16135175     0.11881733     0.00000000     0.00600000   :  2.15728807E-09',
 '    -0.00000000     0.00000000     0.00238949    -0.03894542     1.00000000    -0.00004588   :  3.72127404E-06',
 '     0.00000000     0.00000000     0.00000000     0.00000000     0.00000000     1.00000000   :  1.11022302E-16']

In [7]:
#Well that's not right... set it back and see if it returns to preliminary value
tao.cmd("set ele patch_sector20 e_tot_set=10e9")
tao.cmd(f"show taylor_map -s {zMin} {zMax}")

['Symplectic Error:  6.74E-15',
 '',
 '    -1.41786573     9.92559649     0.00000000     0.00000000     0.00000000    -0.00000000   : -8.45168474E-15',
 '    -0.15970574     0.41271516    -0.00000000     0.00000000     0.00000000     0.00000000   :  1.12084866E-20',
 '     0.00000000    -0.00000000    -1.78592886     5.00259169     0.00000000     0.05592080   :  1.37585823E-09',
 '     0.00000000    -0.00000000    -0.14156116    -0.16340367     0.00000000     0.00600002   :  1.33844543E-15',
 '     0.00000000    -0.00000000     0.00279939    -0.03915330     1.00000000    -0.00000576   :  3.91911853E-12',
 '     0.00000000     0.00000000     0.00000000     0.00000000     0.00000000     1.00000000   : -2.71050543E-20']

In [8]:
#OK, it does. Turns out Bmad tries recalculating the quad settings; that's the problem... it's too "helpful"
tao.cmd("set ele patch_sector20 e_tot_set=10e9")
print(tao.ele_gen_attribs("Q0D")['B1_GRADIENT'])

tao.cmd("set ele patch_sector20 e_tot_set=8e9")
print(tao.ele_gen_attribs("Q0D")['B1_GRADIENT'])


9.79154046334639
7.83323236492439


## Systematic

In [9]:
#Nominal z_ob and z_im
# zMin = 1994.62 - bmadZOffset
# zMax = 2015.26 - bmadZOffset

#Q0D upper and lower bounds
zMin = 997.385254
zMin = 998.385254


In [10]:
#This time, PATCH the energy then set the quads explicitly...
tao=Tao('-init {:s}/bmad/models/f2_elec/tao.init -noplot'.format(environ['FACET2_LATTICE'])) 
print("As imported")
print(tao.ele_gen_attribs("Q0D")['B1_GRADIENT'])
print(tao.ele_gen_attribs("Q0D")['K1'])
tao.cmd(f"show taylor_map -s {zMin} {zMax}")

As imported
10.9694769332638
-0.328856645709


['Symplectic Error:  1.11E-15',
 '',
 '    -7.44028030    13.75736820     0.00000000     0.00000000     0.00000000    -0.00000000   : -9.97509881E-18',
 '    -0.53099539     0.84742763     0.00000000     0.00000000     0.00000000     0.00000000   : -3.40473598E-32',
 '     0.00000000    -0.00000000     0.55155892     7.44930332     0.00000000     0.05590327   :  2.51681292E-16',
 '    -0.00000000     0.00000000    -0.12732662     0.09338147     0.00000000     0.00600002   : -2.99052209E-19',
 '     0.00000000    -0.00000000    -0.01042734    -0.03947563     1.00000000    -0.00000582   : -9.45120061E-17',
 '     0.00000000     0.00000000     0.00000000     0.00000000     0.00000000     1.00000000   :  0.00000000E+00']

In [11]:
setLattice(tao, Q0DkG = -97.9, Q1DkG = 151.7, Q2DkG = -97.9)
tao.cmd("reinit data")

print("Initial magnet assignment")
print(tao.ele_gen_attribs("Q0D")['B1_GRADIENT'])
print(tao.ele_gen_attribs("Q0D")['K1'])
tao.cmd(f"show taylor_map -s {zMin} {zMax}")

Initial magnet assignment
9.78845977900564
-0.293496816765105


['Symplectic Error:  6.66E-16',
 '',
 '    -5.50137032    15.94979181     0.00000000    -0.00000000     0.00000000     0.00000000   : -3.41455806E-17',
 '    -0.40255764     0.98533823     0.00000000     0.00000000     0.00000000     0.00000000   :  8.81112160E-32',
 '     0.00000000    -0.00000000     0.79957511     8.84379054     0.00000000     0.05590324   :  3.34262931E-15',
 '    -0.00000000     0.00000000    -0.09305724     0.22139417     0.00000000     0.00600002   : -2.70484450E-19',
 '    -0.00000000     0.00000000    -0.00999967    -0.04068625     1.00000000    -0.00000582   : -8.00855290E-18',
 '     0.00000000     0.00000000     0.00000000     0.00000000     0.00000000     1.00000000   :  1.97215226E-30']

In [12]:
tao.cmd("set ele patch_sector20 e_tot_set=8e9")
tao.cmd("reinit data")

print("After energy update")
print(tao.ele_gen_attribs("Q0D")['B1_GRADIENT'])
print(tao.ele_gen_attribs("Q0D")['K1'])
tao.cmd(f"show taylor_map -s {zMin} {zMax}")

After energy update
7.83199999424593
-0.293496816765105


['Symplectic Error:  7.44E-15',
 '',
 '    -3.69314424    13.71735004    -0.00000000     0.00000000     0.00000000    -0.00000000   :  4.83499471E-12',
 '    -0.35546696     1.04952975     0.00000000     0.00000000     0.00000000     0.00000000   : -9.26952028E-19',
 '    -0.00000000     0.00000000     1.40097597     9.21529747     0.00000000     0.03578929   :  2.23332009E-03',
 '     0.00000000    -0.00000000    -0.04125108     0.44244801     0.00000000     0.00600000   :  2.15728807E-09',
 '     0.00000000    -0.00000000    -0.00988220    -0.03945687     1.00000000    -0.00004589   :  3.72067032E-06',
 '     0.00000000     0.00000000     0.00000000     0.00000000     0.00000000     1.00000000   :  1.94289029E-16']

In [13]:
setLattice(tao, Q0DkG = -97.9, Q1DkG = 151.7, Q2DkG = -97.9)
tao.cmd("reinit data")

print("Reassert magnets")
print(tao.ele_gen_attribs("Q0D")['B1_GRADIENT'])
print(tao.ele_gen_attribs("Q0D")['K1'])
tao.cmd(f"show taylor_map -s {zMin} {zMax}")

Reassert magnets
9.79
-0.366871021225917


['Symplectic Error:  3.71E-15',
 '',
 '    -5.50311593    12.76164189     0.00000000    -0.00000000     0.00000000     0.00000000   : -4.14227096E-12',
 '    -0.50327356     0.98536848    -0.00000000     0.00000000     0.00000000     0.00000000   : -1.24139336E-17',
 '     0.00000000    -0.00000000     0.79899529     7.07433775     0.00000000     0.03578932   :  2.23331260E-03',
 '     0.00000000    -0.00000000    -0.11637234     0.22120628     0.00000000     0.00600000   :  2.15728807E-09',
 '    -0.00000000     0.00000000    -0.00895886    -0.03452919     1.00000000    -0.00004589   :  3.72066576E-06',
 '     0.00000000     0.00000000     0.00000000     0.00000000     0.00000000     1.00000000   :  1.11022302E-16']

In [16]:
setLattice(tao, Q0DkG = 0)
tao.cmd("show ele Q0D -all")

['Element # 1552',
 'Element Name: Q0D',
 'Element Type:  "2.13Q38.31"',
 'Element Alias:  "LI20:LGPS:3204"',
 'Key: Quadrupole',
 'S_start, S:    997.385254,    998.385254',
 'Ref_time_start, Ref_time:  3.326936E-06,  3.330272E-06',
 '',
 'Attribute values:',
 '    1  L                           =  1.0000000E+00 m',
 '    2  TILT                        =  0.0000000E+00 rad      60  TILT_TOT                    =  0.0000000E+00 rad',
 '    4  K1                          =  0.0000000E+00 1/m^2    45  B1_GRADIENT                 = -0.0000000E+00 T/m',
 '   10  FRINGE_TYPE                 =  None (1)               11  FRINGE_AT                   =  Both_Ends (3)',
 '   13  SPIN_FRINGE_ON              =  T (1)',
 '   16  R0_MAG                      =  0.0000000E+00 m        15  R0_ELEC                     =  0.0000000E+00 m',
 '   17  STATIC_LINEAR_MAP           =  F (0)',
 '   23  FQ1                         =  0.0000000E+00 m        24  FQ2                         =  0.0000000E+00 m',
 ' 